our goal

have each .dat file treated as a single row with 2000 data points (will be resized to 50 x 40)
put each one of these .dat files into a main dataframe
i.e.

(a+c+t+g # of dat, 300) x (2000)

each single datapoint will be resized to a picture, 50x40, we have 300 of them
75 pictures per base

make a labels file, 300 x 4 (one hot encode)

In [2]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F

In [4]:
data = np.zeros((297,2000)) #since _A58 had 1224 points, A_68 also had less substract by 2?
i = a_count = c_count = g_count = t_count = 0
source_dir = Path('DNA/')

for folders in os.listdir("DNA/"):
    print(folders)
    source_dir = Path('DNA/'+folders)
    # absolute_path = os.path.abspath(source_dir)
    # print("Full path: " + absolute_path)
    # print("Directory Path: " + os.path.dirname(absolute_path))

    for raw_data in os.listdir(source_dir):
        data_path = Path('DNA/'+folders+'/'+raw_data)
        # print(i, raw_data, data_path)

        # absolute_path = os.path.abspath(data_path)
        # print("Full path: " + absolute_path)
        # print("Directory Path: " + os.path.dirname(absolute_path))
        # print(np.genfromtxt(data_path, delimiter='\t'))
        # print(np.loadtxt(data_path))
        # print(pd.read_csv(data_path,sep=' ',names = ['time', 'current']))

        data[i,:] = pd.read_csv(data_path,sep=' ',names = ['time', 'current'])['current'][0:2000]
        # print(i, data[i,:])
        
        i += 1
        if folders == 'A':
            a_count += 1
        elif folders == 'C':
            c_count += 1
        elif folders == 'G':
            g_count += 1
        else:
            t_count += 1


print(data)
print(a_count, c_count, g_count, t_count)

labels = np.zeros((297,4))
for j in range(len(labels)):
    if j < a_count:
        labels[j,0] = 1
    elif a_count <= j < (a_count + c_count):
        labels[j,1] = 1
    elif (a_count + c_count) <= j < (a_count + c_count + g_count):
        labels[j,2] = 1
    elif j >= (a_count + c_count + g_count):
        labels[j,3] = 1

print(labels, np.sum(labels[:,0]), np.sum(labels[:,1]), np.sum(labels[:,2]), np.sum(labels[:,3]))

labels2 = np.zeros((297))
for j in range(len(labels)):
    if j < a_count:
        labels2[j] = 0
    elif a_count <= j < (a_count + c_count):
        labels2[j] = 1
    elif (a_count + c_count) <= j < (a_count + c_count + g_count):
        labels2[j] = 2
    elif j >= (a_count + c_count + g_count):
        labels2[j] = 3

A
C
G
T
[[-207.97638578  180.07280633  124.81725379 ... -437.1953854
   206.75177055  -98.57610457]
 [ -67.90753058 -139.75267189 -123.63092518 ...  219.0008219
  -126.9087813  -133.07615075]
 [-200.57816597   74.44650396   23.99452835 ...  113.68002792
  -379.79792376   88.68582645]
 ...
 [ -27.06981436  117.20208853  271.27015217 ...  291.07867222
  -118.45682056   10.35573719]
 [  30.32838415  248.40394     -62.00122146 ... -193.95503378
   217.67315365 -257.97382595]
 [  10.79077708 -353.25044902  129.41285985 ...   94.67509104
   163.16901564 -129.20393461]]
73 75 74 75
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]] 73.0 75.0 74.0 75.0


In [5]:
# x=data[:,0:1600]
x=data[0:-1,200:-200]
labels=labels[0:-1,:]
labels2=labels2[0:-1]


print(x.shape, labels2.shape)

(296, 1600) (296,)


In [6]:
def mean(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.mean(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def median(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.median(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def sig_image(data,size1,size2):
    X=np.zeros((data.shape[0],size1,size2))
    for i in range(data.shape[0]):
        X[i]=(data[i,:].reshape(size1,size2))
    return X.astype(np.float16)

In [7]:
channel_mean=(mean(x,10)).astype(np.float16)
x_m=sig_image(channel_mean,40,40)
channel_median=(median(x,10)).astype(np.float16)
x_md=sig_image(x,40,40)

In [8]:
x_n=sig_image(x,40,40)

In [9]:
x_n.shape

(296, 40, 40)

In [10]:
x_m.shape

(296, 40, 40)

In [11]:
X=np.stack((x_n,x_m,x_md),axis=1).astype(np.float16)

In [12]:
X.shape

(296, 3, 40, 40)

In [13]:
from sklearn.model_selection import train_test_split
trainx, testx, trainlabel, testlabel = train_test_split(X, labels2, test_size=0.2, random_state=20)

print(trainx.shape, testx.shape, trainlabel.shape, testlabel.shape)

(236, 3, 40, 40) (60, 3, 40, 40) (236,) (60,)


In [14]:
sig_train, sig_test = trainx,testx
lab_train, lab_test = trainlabel,testlabel

In [15]:
sig_train = torch.from_numpy(sig_train)
sig_test = torch.from_numpy(sig_test)
lab_train= torch.from_numpy(lab_train)
lab_test = torch.from_numpy(lab_test)

In [16]:
import torch.utils.data as data_utils
batch_size = 2
train_tensor = data_utils.TensorDataset(sig_train, lab_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
print(sig_train[0])

tensor([[[-1.2506e+02,  6.1906e+01, -2.1800e+02,  ...,  1.0281e+02,
           1.3789e+01, -4.3719e+01],
         [ 9.1500e+01, -1.6612e+02,  1.0775e+02,  ...,  2.2500e+01,
           5.0859e+00,  1.2256e+02],
         [-2.5925e+02,  1.4375e+02, -4.2891e+00,  ..., -6.3531e+01,
           1.0806e+02,  9.6438e+01],
         ...,
         [-2.5250e+02,  1.0119e+02, -8.9250e+01,  ...,  2.6828e+01,
          -9.1938e+01, -2.0703e+01],
         [ 5.9141e+00,  4.7000e+01,  3.3594e+01,  ..., -5.9469e+01,
           2.4438e+02, -1.1231e+02],
         [ 1.1062e+02,  3.9031e+01, -2.0275e+02,  ...,  1.5350e+02,
          -5.7438e+01,  2.6312e+01]],

        [[-1.2727e+01,  5.9375e+00, -6.5117e+00,  ...,  1.2129e+00,
           4.0991e-01, -1.1547e+01],
         [-7.7188e+00,  1.4906e+01,  2.3547e+01,  ...,  2.2688e+01,
           8.7969e+00, -3.7871e+00],
         [-3.4531e+01, -7.4492e+00, -1.4023e+00,  ...,  4.9844e+01,
           5.3719e+01,  5.1438e+01],
         ...,
         [-9.6172e+00,  1

In [17]:
batch_size = 3
test_tensor = data_utils.TensorDataset(sig_test, lab_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [18]:
sig_train.size()

torch.Size([236, 3, 40, 40])

In [19]:
sig_test.size()

torch.Size([60, 3, 40, 40])

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3,stride=1,padding = 1)
        self.mp1 = nn.MaxPool2d(kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(32,64, kernel_size=3,stride =1)
        self.mp2 = nn.MaxPool2d(kernel_size=3,stride=1)
        self.fc1= nn.Linear(92416,256)
        self.fc2 = nn.Linear(46208,23104)
        self.fc3 = nn.Linear(23104,11552)
        self.fc4 = nn.Linear(11552, 256)
        # self.dp1 = nn.Dropout(p=0.2)
        self.fc5 = nn.Linear(256,4)

    def forward(self, x):
        in_size = x.size(0)
        # print(in_size)
        x = F.relu((self.conv1(x)))
        # print("step1", x)
        x = F.relu((self.conv2(x)))
        # print("step2", x)
        # print('step2',x.shape)
        x = x.view(in_size,-1)
        # print("step3", x.shape, x)
        x = F.relu(self.fc1(x))
        # print("step4", x)
        # x = self.dp1(x)
        # print("step5", x)
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        x = self.fc5(x)

        return x #F.log_softmax(x, dim=1)

In [21]:
cnn = CNN().double()

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.000001)

In [23]:
num_epochs = 100

In [24]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        print(outputs)
        loss = criterion(outputs, labels.long())
        # print(labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        
        loss.backward()
        optimizer.step()
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list.append(correct / total)

        if (epoch+1) % 5 == 0 or epoch==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

tensor([[ -5.5793,   1.1095,  -6.5323,   1.5765],
        [-11.6472,  -0.8273,  -7.8066,  -1.6623]], dtype=torch.float64,
       grad_fn=<AddmmBackward0>)
Epoch [1/100], Step [1/118], Loss: 4.1473, Train Accuracy: 0.00%
tensor([[-9.1762e+00, -2.5382e+00, -4.7761e+00, -9.9824e+00],
        [-7.7912e+00, -1.4705e+00, -7.0357e+00,  6.8461e-03]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)
Epoch [1/100], Step [2/118], Loss: 5.1728, Train Accuracy: 0.00%
tensor([[-3.4990, -0.7954, -7.6448, -9.6389],
        [-3.4023, -0.8468,  1.4224, -6.8181]], dtype=torch.float64,
       grad_fn=<AddmmBackward0>)
Epoch [1/100], Step [3/118], Loss: 3.5106, Train Accuracy: 50.00%
tensor([[-3.3707, -5.1713, -2.1183, -1.6513],
        [-4.9483, -5.5316, -3.0912, -7.1522]], dtype=torch.float64,
       grad_fn=<AddmmBackward0>)
Epoch [1/100], Step [4/118], Loss: 1.8731, Train Accuracy: 0.00%
tensor([[-6.0403, -4.5912, -5.7313, -5.8944],
        [-2.4516, -0.6719,  4.0593, -5.0477]], dtype=torch.float6

KeyboardInterrupt: 

In [ ]:
total_step = len(test_loader)
print(total_step)
loss_list_test = []
acc_list_test = []
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        if epoch%10 ==0:
            print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

In [ ]:
# if you need to save
torch.save(cnn,'cnnTC3_fold3_45.pth')